In [ ]:
!pip freeze
!pip3 install numpy
!pip3 install pandas
!pip3 install sklearn
!pip3 install matplotlib
!pip3 install pydotplus
!pip3 install six

Problem 2: Decisions, decisions

Implement ID3 from scratch

Below is the prep and helper functions.

In [1]:
import pandas as pd
import numpy as np

In [64]:
# preparing data for use
url = 'https://raw.githubusercontent.com/f1nn3g4n/SENG474Test/ad161e4c364f92adc9730fe6c06edd2769b9f355/elections_clean.csv'
df = pd.read_csv(url, sep='\t')
# help from https://datascienceparichay.com/article/randomly-shuffle-pandas-dataframe-rows/#:~:text=There%20are%20a%20number%20of%20ways%20to%20shuffle,sklearn.utils%20to%20shuffle%20your%20dataframe.%20Here%E2%80%99s%20the%20syntax%3A
df = df.sample(frac=1)

total = df.shape[0]
train_size = int(round(total*0.7))

df_train = df.iloc[:train_size,:]
df_valid = df.iloc[train_size:,:]

In [25]:
category_features = ['ethnic_male', 'ethnic_female', 'state', 'county',
                    'education',]
label_vector = 'democrat'
values = df_train[label_vector].unique()

In [ ]:
# https://medium.com/geekculture/step-by-step-decision-tree-id3-algorithm-from-scratch-in-python-no-fancy-library-4822bbfdd88f

In [66]:
def findInfoGain(feature, train_data):
  ''' Takes a feature and training data and finds information gain, 
  for feature splitting. 
  '''
  feature_vals = feature.unique()
  n = train_data.shape[0]
  info_gain = 0
  feature_val_counts = feature.value_counts(sort=False)
  tree_entropy = 0
  tree_ent_flag = False
  
  # feature entropy
  for feat_val, count in feature_val_counts.iteritems():
    f_data = train_data[train_data[feature.name]==feat_val]
    entropy = 0
    for value in values:
      if not tree_ent_flag:
        # two time tree entropy calculation
        tree_ent_flag = tree_entropy != 0
        t_val_count = train_data[train_data[label_vector]==value].shape[0]
        t_prob = t_val_count/n
        tree_entropy += -t_prob*np.log2(t_prob)

      val_count = f_data[f_data[label_vector]==value].shape[0]
      if(count == np.nan):
        print(feat_val)
      prob = val_count/count
      entropy += -prob*np.log2(prob)
    info_gain += count/n * entropy

  return tree_entropy - info_gain

In [63]:
def findSplitFeature(train_data):
  max_ig = -1
  max_info_feature = None

  for category in category_features:
    category_ig = findInfoGain(df_train[category], df_train)
    if max_ig < category_ig:
      max_ig = category_ig
      max_info_feature = df_train[category]

  return max_info_feature

In [65]:
print(findSplitFeature(df_train).name)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log2
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


education


In [67]:
print(findInfoGain(df_train['county'],df_train))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in log2
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars


nan


Below is the tree building function and resulting tree.

In [ ]:
class TreeNode:
  def __init__(self, data):
    self.data = data # where data is a pandas Series
    self.children = {} # dictionary of value:node pairs
    for value in data.unique():
      self.children[value] = None

In [ ]:
def buildID3Tree(train_data):
  ''' Takes in training data, returns a tree using the TreeNode class above. '''
  root_feature = findSplitFeature(train_data)
  root = TreeNode(root_feature)
  # if only one example or prob(dem) ==1 for all examples, child should be 1 or 
  # 0, depending on democrat
  # else, should be another node

In [33]:
print(findSplitFeature(df_train).name)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log2
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


2203    LT High School
2971    LT High School
314       Some College
2544    GTE Bachelor's
890     LT High School
             ...      
2371    LT High School
2034    LT High School
1597    GTE Bachelor's
2966       High School
2357    GTE Bachelor's
Name: education, Length: 2198, dtype: object


In [ ]:
class_data = 